# Richter's Predictor: Modeling Earthquake Damage

Hosted By _DrivenData_ at ['Richter's Predictor: Modeling Earthquake Damage'](https://www.drivendata.org/competitions/57/nepal-earthquake/).

The dataset mainly consists of information on the buildings' structure and their legal ownership. Each row in the dataset represents a specific building in the region that was hit by the Gorkha earthquake.

We're trying to predict the ordinal variable `damage_grade`, which represents a level of damage to the building that was hit by the earthquake. There are 3 grades of the damage:

 1. represents low damage
 2. represents a medium amount of damage
 3. represents almost complete destruction

The level of damage is an ordinal variable meaning that ordering is important. This can be viewed as a classification or an ordinal regression problem. 
 
To measure the performance of our algorithms, we'll use the _F1 score_ which balances the precision and recall of a classifier. Traditionally, the F1 score is used to evaluate performance on a binary classifier, but since we have three possible labels we will use a variant called the _micro averaged F1 score_.
 
 - [Loading data](#Loading-data)
 - [Exploratory data analysis](#Exploratory-data-analysis)
 - [Correlation](#Correlation)
 - [Model training](#Model-training)
 - [Grid search with cross validation](#Grid-search-with-cross-validation)
    - [XGB Classifier](#XGB-Classifier)
 - [Ensemble modeling](#Ensemble-modeling)
 - [Performance metric for DrivenData competition](#Performance-metric-for-DrivenData-competition)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path, PurePath
from time import time
%matplotlib inline

In [ ]:
pd.options.display.max_columns = None

## Loading data

In [2]:
project_root_dir = Path('/Users/angelo/Programming/Python/driven-data/predicting-earthquake-damage')
train_values_file = project_root_dir / 'data/train_values.csv'
train_labels_file = project_root_dir / 'data/train_labels.csv'
test_values_file = project_root_dir / 'data/test_values.csv'

In [3]:
train_values_df = pd.read_csv(train_values_file, index_col='building_id')
test_values_df = pd.read_csv(test_values_file, index_col='building_id')
train_labels_df = pd.read_csv(train_labels_file, index_col='building_id')

In [ ]:
print(f'train_values_df.shape: {train_values_df.shape}\ntrain_labels_df.shape: {train_labels_df.shape}\n')

In [ ]:
print(f'train_values_df.index.nunique(): {train_values_df.index.nunique()}\ntrain_labels_df.index.nunique(): {train_labels_df.index.nunique()}')

In [ ]:
print(f'(train_values_df.index.sort_values() == train_labels_df.index.sort_values()).all(): \
{(train_values_df.index.sort_values() == train_labels_df.index.sort_values()).all()}')

In [4]:
all_dataframes = [train_values_df, test_values_df]
all_regexes = ['geo_level_']
for df in all_dataframes:
    for reg in all_regexes:
        filter_ = train_values_df.filter(regex=reg).columns
        df[filter_] = df[filter_].astype('category')

In [5]:
other_objects = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', \
                 'other_floor_type', 'position',  'plan_configuration', 'legal_ownership_status']
for df in all_dataframes:
    df[other_objects] = df[other_objects].astype('category')

In [ ]:
# for df in all_dataframes:
#     df.drop(['geo_level_2_id', 'geo_level_3_id'], axis='columns', inplace=True)

In [6]:
num_attrib = train_values_df.select_dtypes('int64').columns
cat_attrib = train_values_df.select_dtypes('category').columns

In [7]:
cat_attrib

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status'],
      dtype='object')

In [8]:
num_attrib

Index(['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other'],
      dtype='object')

In [9]:
train_values_df[num_attrib] = train_values_df[num_attrib].astype('int32')
test_values_df[num_attrib] = test_values_df[num_attrib].astype('int32')
train_labels_df['damage_grade'] = train_labels_df['damage_grade'].astype('category')

In [ ]:
train_values_df.shape

In [10]:
train_values_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  category
 1   geo_level_2_id                          260601 non-null  category
 2   geo_level_3_id                          260601 non-null  category
 3   count_floors_pre_eq                     260601 non-null  int32   
 4   age                                     260601 non-null  int32   
 5   area_percentage                         260601 non-null  int32   
 6   height_percentage                       260601 non-null  int32   
 7   land_surface_condition                  260601 non-null  category
 8   foundation_type                         260601 non-null  category
 9   roof_type                               260601 non-null  category
 10  ground_floor_type          

In [26]:
for col in list(train_values_df.columns):
    print(f'{col}: {train_values_df[col].unique()}')

geo_level_1_id: [6, 8, 21, 22, 11, ..., 24, 28, 23, 2, 29]
Length: 31
Categories (31, int64): [6, 8, 21, 22, ..., 28, 23, 2, 29]
geo_level_2_id: [487, 900, 363, 418, 131, ..., 975, 771, 77, 115, 627]
Length: 1414
Categories (1414, int64): [487, 900, 363, 418, ..., 771, 77, 115, 627]
geo_level_3_id: [12198, 2812, 8973, 10694, 1488, ..., 7039, 8947, 3152, 5276, 3085]
Length: 11595
Categories (11595, int64): [12198, 2812, 8973, 10694, ..., 8947, 3152, 5276, 3085]
count_floors_pre_eq: [2 3 1 4 5 6 7 8 9]
age: [ 30  10  25   0  15  20  45  55   5  40  80  60  35  70  50  65 100  75
  85 190 995 105  90 120  95 110 115 150 200 130 125 140 155 160 175 135
 145 195 180 165 170 185]
area_percentage: [  6   8   5   9   3  13   7   4  12  16  11  27  10  15  14  17  21  37
  19   2  28  38  56   1  20  24  34  26  18  31  25  23  22  32  47  36
  40  29  42  55  35  39 100  50  51  43  30  62  85  33  45  52  57  49
  67  66  54  75  65  58  48  64  63  46  59  86  78  41  44  61  70  77
  73  72

In [25]:
print(f'''land_surface_condition: {[type(item) for item in train_values_df['land_surface_condition'].unique()]}''')

land_surface_condition: [<class 'str'>, <class 'str'>, <class 'str'>]


In [ ]:
train_values_df.head()

In [ ]:
train_labels_df.info()

In [ ]:
train_labels_df.head()

In [ ]:
train_labels_df.describe()

In [ ]:
type(train_labels_df['damage_grade'].values[1])

In [ ]:
train_values_df.nunique().sort_values(ascending=False)

In [ ]:
train_values_df.describe().applymap('{:.2f}'.format)

In [ ]:
test_values_df.info()

In [ ]:
test_values_df.head()

In [ ]:
test_values_df.nunique().sort_values(ascending=False)

## Exploratory data analysis

In [ ]:
mod_num_attrib_list = list(num_attrib)
mod_num_attrib_list.append(train_labels_df.columns[-1])
mod_num_attrib_df = train_values_df.join(train_labels_df)[mod_num_attrib_list]

In [ ]:
mod_num_attrib_df.head()

In [34]:
plot_dir = project_root_dir / 'exploratory_data_analysis/plots'
model_dir = project_root_dir / 'models'

In [ ]:
from matplotlib import ticker
scale = 1e4
ticks = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale))

In [ ]:
# sns.set(style='ticks', color_codes=True)
fig, axes = plt.subplots(figsize=(10, 6))
df = train_labels_df['damage_grade'].value_counts(ascending=False).sort_index()
axes.bar(df.index, df.values, color=plt.cm.tab10.colors, edgecolor='k')
axes.set_xlabel('Level', fontsize=14)
axes.set_ylabel('Number of damaged buildings (in units of $10^4$)', fontsize=14)
axes.set_title('Building damage by level', fontsize=16)
axes.set_xticks(df.index)
axes.set_xticklabels(['Low', 'Medium', 'High'], fontsize=14)
plt.setp(axes.get_yticklabels(), fontsize=14)
axes.yaxis.set_major_formatter(ticks)
plt.savefig(PurePath.joinpath(plot_dir, 'damage-level-by-grade.png'), dpi=288);

In [ ]:
t0 = time()
label_font_size = 14
pairplot_file = PurePath.joinpath(plot_dir, 'pairplot.png')
if pairplot_file.is_file():
    from IPython.display import Image
    display(Image(filename = pairplot_file))
else:
    g = sns.PairGrid(mod_num_attrib_df, hue='damage_grade', palette='tab10', height=3.0, aspect=1.2)
    g = g.map_diag(plt.hist)
    g = g.map_offdiag(plt.scatter, s=60, alpha=0.6)
    g = g.add_legend()
#     print(g._legend.__dir__())
#     g._legend._set_loc('center left')
#     g._legend.set_bbox_to_anchor((1, 0.5, 1, 2))
    g._legend.set_title('Damage Level', prop={'size': label_font_size})
    for txt, lb in zip(g._legend.texts, ['Low', 'Medium', 'High']):
        txt.set_text(lb)
        txt.set_fontsize(label_font_size)

    xlabels, ylabels = [], []

    for ax in g.axes[-1,:]:
        xlabel = ax.xaxis.get_label_text()
        xlabels.append(xlabel)
    for ax in g.axes[:,0]:
        ylabel = ax.yaxis.get_label_text()
        ylabels.append(ylabel)
        
    for i in range(len(xlabels)):
        for j in range(len(ylabels)):
            g.axes[j, i].xaxis.set_label_text(xlabels[i])
            g.axes[j, i].xaxis.label.set_size(label_font_size)
            g.axes[j, i].tick_params(axis='x', which='major', labelsize=label_font_size)
            g.axes[j, i].yaxis.set_label_text(ylabels[j])
            g.axes[j, i].yaxis.label.set_size(label_font_size)
            g.axes[j, i].tick_params(axis='y', which='major', labelsize=label_font_size)

    plt.tight_layout(rect=(0, 0, 0.92, 1))
    plt.savefig(pairplot_file, dpi=288)
print(f'Time elasped: {time() - t0:.4f} sec');

In [ ]:
print(f'Categorical columns: {list(cat_attrib)}\n')
print(f'Number of categorical columns: {len(cat_attrib)}\n')

binary_cat_attrib = list(train_values_df.filter(regex='has_').columns)
geo_id_cat_attrib = list(train_values_df.filter(regex='geo_level_').columns)
print(f'Number of binary categorical columns: {len(binary_cat_attrib)}\n')

list_tuple_binary_cat_attrib = [(binary_cat_attrib[i], binary_cat_attrib[j]) for i, j in \
                                zip(range(0, len(binary_cat_attrib)-1, 2), range(1, len(binary_cat_attrib), 2))]

print(f'List of tuples: {list_tuple_binary_cat_attrib}\n')
print(f'Number of tuples: {len(list_tuple_binary_cat_attrib)}\n')

list_remainders = list(set(cat_attrib) - set(binary_cat_attrib) - set(geo_id_cat_attrib))
print(f'Remainder columns: {list(list_remainders)}\n')
print(f'Number of reminder columns: {len(list_remainders)}\n')

list_tuple_remainders = [(list_remainders[i], list_remainders[j]) for i, j in \
                         zip(range(0, len(list_remainders)-1, 2), range(1, len(list_remainders), 2))]
print(f'List tuple remainders: {list_tuple_remainders}')

In [ ]:
def bar_plot(ax, df, col):
    for i in range(0, 2):
        df_col = df[col[i]].value_counts(ascending=False)
        df_col = df_col/df_col.sum()*100
        df_col = df_col.reset_index().rename(columns={'index': 'features'})
        ax[i].bar(df_col['features'], df_col[col[i]], color=colors, edgecolor='k')        
        if all(isinstance(tick, np.float64) for tick in ax[i].get_xticks()):
            ax[i].set_xticks(range(0, 2))
            ax[i].set_xticklabels(['No', 'Yes'])
        plt.setp(ax[i].get_xticklabels(), ha="right", rotation_mode="anchor", rotation=0, fontsize=14)
        plt.setp(ax[i].get_yticklabels(), fontsize=14)
        ax[i].set_ylabel('Percent (%)', fontsize=14)
        ax[i].set_title(col[i], fontsize=16);

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(14, 12))
colors = plt.cm.tab10.colors
for ax, col in zip(axes, list_tuple_remainders):
    bar_plot(ax, train_values_df, col)
plt.tight_layout()
plt.savefig(PurePath.joinpath(plot_dir, 'categorical-bar-plot-1.png'), dpi=288);

In [ ]:
fig, axes = plt.subplots(nrows=11, ncols=2, figsize=(14, 30))
for ax, col in zip(axes, list_tuple_binary_cat_attrib):
    bar_plot(ax, train_values_df, col)
plt.tight_layout()
plt.savefig(PurePath.joinpath(plot_dir, 'categorical-bar-plot-2.png'), dpi=288);

## Correlation

In [ ]:
fig, axes = plt.subplots(figsize=(10, 6))
mask = np.zeros_like(train_values_df[num_attrib].corr())
mask[np.tril_indices_from(mask)] = True
hm = sns.heatmap(data=train_values_df[num_attrib].corr(), cmap='Spectral', ax=axes, annot=True, fmt='1.4f', mask=mask, annot_kws={'size': 14})
axes.tick_params(labelsize=14)
cbar = hm.collections[0].colorbar
cbar.ax.tick_params(labelsize=14)
plt.setp(axes.get_xticklabels(), ha='right', rotation_mode='anchor', rotation=45)
plt.setp(axes.get_yticklabels(), ha='right', rotation_mode='anchor')
plt.tight_layout()
plt.grid(True, linestyle='--')
plt.savefig(PurePath.joinpath(plot_dir, 'correlation.png'), dpi=288);

## Model training

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from xgboost import XGBClassifier
from joblib import dump, load
from collections import namedtuple

In [28]:
train_labels_df_copy = train_labels_df.copy

In [ ]:
type(train_labels_df[['damage_grade']])

In [29]:
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('std_scaler', StandardScaler())])
full_pipeline = ColumnTransformer([('num', num_pipeline, num_attrib), ('cat', OneHotEncoder(), cat_attrib)])
train_values_prepared_df = full_pipeline.fit_transform(train_values_df)
# label_pipeline = OrdinalEncoder()
# train_labels_prepared_df = label_pipeline.fit_transform(train_labels_df)
X_train, X_test, y_train, y_test = train_test_split(train_values_prepared_df, train_labels_df, test_size = 0.3, random_state=42)
y_train, y_test = y_train.iloc[:,0], y_test.iloc[:,0]

In [30]:
print(f'X_train.shape: {X_train.shape:}\nX_test.shape: {X_test.shape}\n\
y_train.shape: {y_train.shape}\ny_test.shape: {y_test.shape}\n')

X_train.shape: (182420, 13105)
X_test.shape: (78181, 13105)
y_train.shape: (182420,)
y_test.shape: (78181,)



In [31]:
clf_ntuple = namedtuple('clf_ntuple', ['name', 'classifier'])
# lin_svc_clf = clf_ntuple(name='lin_svc_clf', classifier=LinearSVC(dual=False, random_state=42))
lr_clf = clf_ntuple(name='lr_clf', classifier=LogisticRegression(random_state=42, n_jobs=-1))
# dt_clf = clf_ntuple(name='dt_clf', classifier=DecisionTreeClassifier(random_state=42))
# rf_clf = clf_ntuple(name='rf_clf', classifier=RandomForestClassifier(random_state=42, n_jobs=-1))
xgb_clf = clf_ntuple(name='xgb_clf', classifier=XGBClassifier(n_jobs=-1))
# clf_tuple = (lin_svc_clf, lr_clf, dt_clf, rf_clf, xgb_clf)
clf_tuple = (lr_clf, xgb_clf)

In [32]:
def print_accuracy(model_clf, t, X_test, y_test):
    y_pred = model_clf.predict(X_test)
    acc_score = accuracy_score(y_pred, y_test)
    print(f'Accuracy score for {model_clf.__class__.__name__}: {acc_score:.6f}', end=', ')
    print(f'time elapsed: {time() - t:.4f} sec')
    print(f'\nClassification report: {classification_report(y_pred, y_test)}')
    return acc_score

In [35]:
print(model_dir)

/Users/angelo/Programming/Python/driven-data/predicting-earthquake-damage/models


In [38]:
t0 = time()
list_files = [x for x in model_dir.iterdir() if x.is_file]
accuracy_score_dict = dict()
for tup in clf_tuple:
    model_file = PurePath.joinpath(model_dir, tup.name+'.sav')
    if model_file in list_files:
        t1 = time()
        model_clf = load(model_file)
        accuracy_score_dict[model_clf.__class__.__name__] = print_accuracy(model_clf, t1, X_test, y_test)
    else:
        t2 = time()
        tup.classifier.fit(X_train, y_train)
        dump(tup.classifier, model_file)
        accuracy_score_dict[tup.classifier.__class__.__name__] = print_accuracy(tup.classifier, t2, X_test, y_test)
print(f'\nTotal time elasped: {time() - t0:.4f} sec')

Accuracy score for LogisticRegression: 0.734143, time elapsed: 0.0248 sec

Classification report:               precision    recall  f1-score   support

           1       0.46      0.67      0.54      5244
           2       0.84      0.73      0.78     50866
           3       0.63      0.75      0.68     22071

    accuracy                           0.73     78181
   macro avg       0.64      0.72      0.67     78181
weighted avg       0.76      0.73      0.74     78181

Accuracy score for XGBClassifier: 0.725995, time elapsed: 1.9924 sec

Classification report:               precision    recall  f1-score   support

           1       0.45      0.71      0.55      4816
           2       0.87      0.71      0.78     53849
           3       0.57      0.76      0.65     19516

    accuracy                           0.73     78181
   macro avg       0.63      0.73      0.66     78181
weighted avg       0.77      0.73      0.74     78181


Total time elasped: 2.1822 sec


## Grid search with cross validation

### XGB Classifier

In [ ]:
import dask.dataframe as dd
train_values_dask_df = dd.from_pandas(train_values_df, npartitions=6)
train_labels_dask_df = dd.from_pandas(train_labels_df, npartitions=6)
test_values_dask_df = dd.from_pandas(test_values_df, npartitions=6)

In [ ]:
# train_values_dask_df = train_values_dask_df.drop(['geo_level_2_id', 'geo_level_3_id'], axis=1)
# test_values_dask_df = test_values_dask_df.drop(['geo_level_2_id', 'geo_level_3_id'], axis=1)

In [ ]:
from dask_ml.xgboost import XGBClassifier
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=2, memory_limit='12GB')
xgb_clf = XGBClassifier()
xgb_clf.fit(train_values_dask_df, train_labels_dask_df)
# xgb_params = {'max_depth': [50, 100], 'n_estimators':[100, 200], 'n_jobs': [-1]}
# xgb_grid_search = GridSearchCV(xgb_clf, xgb_params, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
# xgb_joblib_file = PurePath.joinpath(model_dir, 'xgb_grid_search.sav')

In [ ]:
def grid_search_func(grid_search, joblib_file):
    t0 = time()
    if joblib_file.is_file():
        grid_search = load(joblib_file)
    else:
        grid_search.fit(X_train, y_train)
        dump(grid_search, joblib_file)
    print(f'Best parameters for grid search: {grid_search.best_params_}\n')
    print(f'Best estimator for grid search: {grid_search.best_estimator_}\n')
    print(f'Time elapsed: {time() - t0:.4f} sec')
    return grid_search

In [ ]:
xgb_grid_search_output = grid_search_func(xgb_grid_search, xgb_joblib_file)

In [ ]:
def grid_results(grid_search, num):
    cvres = grid_search.cv_results_
    best_fit_models = [(np.sqrt(-mean_score), params) for mean_score, params in zip(cvres['mean_test_score'], cvres['params'])]
    best_fit_models.sort(key=lambda x: x[0], reverse=False)
    print(f'List of best-fit models sorted by RMSE:')
    for rmse, params in best_fit_models[:num]:
        print(f'{rmse} {params}')

In [ ]:
grid_results(lin_svc_grid_search_output, 8)

In [ ]:
y_pred = lin_svc_grid_search_output.predict(X_test)
best_fit_acc_score = accuracy_score(y_test, y_pred)
print(f'Accuracy for the best-fit model: {best_fit_acc_score:.8f}')

In [ ]:
print(f'''We have a {(100*(best_fit_acc_score-accuracy_score_dict['XGBClassifier'])/accuracy_score_dict['XGBClassifier']):.4f}% improvement.''')

## Ensemble modeling

In [ ]:
best_est_clf = clf_ntuple(name='grid_search.best_estimator_', classifier=xgb_grid_search_output.best_estimator_)
lin_svc_clf = clf_ntuple(name='lin_svc_clf', classifier=LinearSVC(C=1e3, dual=False, random_state=42))
new_clf_tuple = (lin_svc_clf, lr_clf, dt_clf, rf_clf, best_est_clf)
tupl_list = [(tupl.name, tupl.classifier) for tupl in new_clf_tuple]

In [ ]:
voting_clf = VotingClassifier(estimators=tupl_list, voting='hard', n_jobs=-1)

In [ ]:
t0 = time()
joblib_file = PurePath.joinpath(model_dir, 'voting_clf.sav')
if joblib_file.is_file():
    voting_clf = load(joblib_file)
else:
    voting_clf.fit(X_train, y_train)
    dump(voting_clf, joblib_file)
print(f'Time elapsed: {time() - t0:.4f} sec')

In [ ]:
t0 = time()
y_prediction_voting_clf = voting_clf.predict(X_test)
accuracy_score_voting_clf = accuracy_score(y_prediction_voting_clf, y_test)
print(f'Time elapsed: {time() - t0:.4f} sec')

In [ ]:
accuracy_score_dict.update({'GridSearchCV': best_fit_acc_score, 'VotingClassifier': accuracy_score_voting_clf})
sorted_acc_score = [(val, key) for key, val in accuracy_score_dict.items()]

In [ ]:
sorted_acc_score.sort(key=lambda val: val)

In [ ]:
for score, name in sorted_acc_score:
    print(f'''Accuracy score for {name + ':':<25} {score:.6f}''')

## Performance metric for DrivenData competition

In [ ]:
print(f'''Micro-averaged F1 score for the VotingClassifier classifier: {f1_score(y_test, y_pred, average='micro'):.8f}''')